In [ ]:
# ISR is already installed - skip this cell if already installed
# Uncomment below if you need to reinstall ISR
# !pip install ISR==2.2.0 --no-deps
# !pip install imageio tqdm pyaml

# Suppress typing_extensions warning for kernel
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='typing_extensions')

print("ISR is already installed. Skipping installation.")

: 

In [ ]:
import os
import urllib.request
from pathlib import Path

# Create div2k directory if it doesn't exist
os.makedirs('div2k', exist_ok=True)

# Download DIV2K dataset files
dataset_files = {
    'DIV2K_train_LR_bicubic_X2.zip': 'http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_LR_bicubic_X2.zip',
    'DIV2K_valid_LR_bicubic_X2.zip': 'http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_valid_LR_bicubic_X2.zip',
    'DIV2K_train_HR.zip': 'http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_HR.zip',
    'DIV2K_valid_HR.zip': 'http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_valid_HR.zip'
}

for filename, url in dataset_files.items():
    filepath = os.path.join('div2k', filename)
    if not os.path.exists(filepath):
        print(f'Downloading {filename}...')
        try:
            urllib.request.urlretrieve(url, filepath)
            print(f'✓ {filename} downloaded successfully')
        except Exception as e:
            print(f'✗ Error downloading {filename}: {e}')
    else:
        print(f'✓ {filename} already exists')


In [ ]:
import zipfile
import os

# Extract all zip files in div2k directory
zip_files = [
    'div2k/DIV2K_train_LR_bicubic_X2.zip',
    'div2k/DIV2K_valid_LR_bicubic_X2.zip',
    'div2k/DIV2K_train_HR.zip',
    'div2k/DIV2K_valid_HR.zip'
]

for zip_path in zip_files:
    if os.path.exists(zip_path):
        print(f'Extracting {zip_path}...')
        try:
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall('div2k')
            print(f'✓ {zip_path} extracted successfully')
        except Exception as e:
            print(f'✗ Error extracting {zip_path}: {e}')
    else:
        print(f'✗ {zip_path} not found, skipping extraction')


In [ ]:
from ISR.models import RRDN
from ISR.models import Discriminator
from ISR.models import Cut_VGG19


In [ ]:
lr_train_patch_size = 50
layers_to_extract = [5, 9]
scale = 4  # Changed to 4 to match your X4 LR images (your dataset has X4, not X2)
hr_train_patch_size = lr_train_patch_size * scale

rrdn  = RRDN(arch_params={'C':4, 'D':3, 'G':64, 'G0':64, 'T':10, 'x':scale}, patch_size=lr_train_patch_size)
f_ext = Cut_VGG19(patch_size=hr_train_patch_size, layers_to_extract=layers_to_extract)
discr = Discriminator(patch_size=hr_train_patch_size, kernel_size=3)


In [ ]:
from ISR.train import Trainer
import os

loss_weights = {
  'generator': 0.0,
  'feature_extractor': 0.0833,
  'discriminator': 0.01
}
losses = {
  'generator': 'mae',
  'feature_extractor': 'mse',
  'discriminator': 'binary_crossentropy'
} 

log_dirs = {'logs': './logs', 'weights': './weights'}

learning_rate = {'initial_value': 0.0004, 'decay_factor': 0.5, 'decay_frequency': 30}

flatness = {'min': 0.0, 'max': 0.15, 'increase': 0.01, 'increase_frequency': 5}

# ============================================================================
# DATASET PATHS - Automatically configured based on your folder structure
# ============================================================================
# Based on your actual folder structure on E: drive

base_path = 'E:\\'

# Training dataset paths (configured from your actual folders)
# NOTE: Your Train LR folder is X4, but model expects X2. You may need to adjust scale parameter
# or find/create X2 LR images for proper training
lr_train_path = os.path.join(base_path, 'DIV2K_train_LR_bicubic_X4') + os.sep  # Currently X4, needs X2 for scale=2
hr_train_path = os.path.join(base_path, 'DIV2K_train_HR_bicubic_X2', 'DIV2K_train_HR_bicubic', 'X2') + os.sep

# Validation dataset paths
lr_valid_path = os.path.join(base_path, 'DIV2K_valid_LR_bicubic', 'X4') + os.sep  # Currently X4, needs X2 for scale=2
hr_valid_path = os.path.join(base_path, 'DIV2K_valid_HR') + os.sep

# ============================================================================
# IMPORTANT: You have X4 LR images but the model is configured for X2 scale
# Options:
# 1. Change scale parameter to 4 (in Cell 4: scale = 4)
# 2. Or download/get X2 LR images and update paths above
# ============================================================================

# Verify all paths exist
print("Checking dataset directories...")
print(f"Base path: {base_path}\n")

paths_to_check = [
    ('Train LR', lr_train_path),
    ('Train HR', hr_train_path),
    ('Valid LR', lr_valid_path),
    ('Valid HR', hr_valid_path)
]

all_paths_exist = True
for name, path in paths_to_check:
    if path and os.path.exists(path.rstrip(os.sep)):
        print(f'✓ Found: {name}')
        print(f'  Path: {path}')
    else:
        print(f'✗ Missing: {name}')
        print(f'  Path: {path if path else "Not set"}')
        all_paths_exist = False

if not all_paths_exist:
    print("\n⚠️  ERROR: Some dataset directories are missing!")
    print("\nPlease update the paths at the top of this cell to match your dataset locations.")
    print("\nRequired folders:")
    print("  1. Training LR (Low Resolution) images folder")
    print("  2. Training HR (High Resolution) images folder")
    print("  3. Validation LR images folder")
    print("  4. Validation HR images folder")
    raise FileNotFoundError("Dataset directories not found. Please update the paths in this cell.")

print("\n✓ All dataset paths verified!")

trainer = Trainer(
    generator=rrdn,
    discriminator=discr,
    feature_extractor=f_ext,
    lr_train_dir=lr_train_path,
    hr_train_dir=hr_train_path,
    lr_valid_dir=lr_valid_path,
    hr_valid_dir=hr_valid_path,
    loss_weights=loss_weights,
    learning_rate=learning_rate,
    flatness=flatness,
    dataname='div2k',
    log_dirs=log_dirs,
    weights_generator=None,
    weights_discriminator=None,
    n_validation=50,
)

print("\n✓ Trainer initialized successfully!")

In [ ]:
trainer.train(
    epochs=2,
    steps_per_epoch=20,
    batch_size=4,
    monitored_metrics={'val_generator_PSNR_Y': 'max'}
)
